In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset
def build_dataset(words):
    block_size = 3 # context length: how many characters do we take to predict the next one
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->' , itos[ix])
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
Xtr[:3],Ytr[:3]

(tensor([[ 0,  0,  0],
         [ 0,  0, 25],
         [ 0, 25, 21]]),
 tensor([25, 21,  8]))

In [8]:
C = torch.randn(27,2)

In [9]:
emb = C[Xtr]

In [10]:
emb.shape

torch.Size([182625, 3, 2])

In [11]:
W1 = torch.randn(6,100)
b1 = torch.randn(100)

In [12]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [13]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)

In [14]:
logits = h @ W2 + b2

In [15]:
logits.shape

torch.Size([182625, 27])

In [16]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
prob[0].sum()

tensor(1.)

In [18]:
loss = -prob[torch.arange(Xtr.shape[0]),Ytr].log().mean()
loss

tensor(17.1949)

In [20]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator = g)
W1 = torch.randn((6,100), generator = g)
b1 = torch.randn(100, generator = g)
W2 = torch.randn((100,27), generator = g)
b2 = torch.randn(27, generator = g)
parameters = [C,W1,b1,W2,b2]

In [21]:
sum(p.nelement() for p in parameters)

3481

In [22]:
for p in parameters:
    p.requires_grad = True

In [23]:
i = 0
for p in parameters:
    if i < 5:
        #print(f'{p.requires_grad=}')
        #print(f'{p.grad=}')
        #print(f'{p.data=}')
        print(f'{p.data.shape=}')
    i += 1
    if i == 5:
        break

p.data.shape=torch.Size([27, 2])
p.data.shape=torch.Size([6, 100])
p.data.shape=torch.Size([100])
p.data.shape=torch.Size([100, 27])
p.data.shape=torch.Size([27])


In [32]:
for _ in range(10000):
    # construct minibatch
    ix = torch.randint(0,Xtr.shape[0], (32,))
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh( emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    #counts = logits.exp()
    #prob = counts / counts.sum(0, keepdim=True)
    #loss = - prob[torch.arange(31),Ytr].log().mean()
    loss = F.cross_entropy(logits, Ytr[ix])
    # bacward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item())

2.5835630893707275


In [33]:
emb = C[Xtr]
h = torch.tanh( emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.4593, grad_fn=<NllLossBackward0>)

In [34]:
emb = C[Xdev]
h = torch.tanh( emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.4608, grad_fn=<NllLossBackward0>)

In [35]:
logits[:32].max(1)

torch.return_types.max(
values=tensor([4.7208, 4.4151, 5.0162, 2.2739, 2.7659, 4.7208, 4.4151, 2.6560, 3.0382,
        4.2985, 4.2372, 3.4206, 4.7208, 6.0500, 1.6402, 3.7205, 2.8900, 3.1819,
        2.2340, 4.7208, 6.6244, 2.2631, 1.9860, 2.2805, 1.6192, 4.7208, 5.8904,
        1.5248, 3.1645, 3.4219, 4.7208, 6.8819], grad_fn=<MaxBackward0>),
indices=tensor([11, 12,  5,  0, 12, 11, 12,  1,  0,  0,  0,  0, 11,  1, 19,  0,  0,  0,
         0, 11,  1,  5, 12,  0,  0, 11,  1, 19,  0,  5, 11,  1]))

In [41]:
Ytr[:32]

tensor([25, 21,  8,  5, 14,  7,  0,  4,  9, 15, 14,  4, 18,  5,  0, 24,  1, 22,
         9,  5, 14,  0, 10, 15, 18,  9,  0, 10, 21,  1, 14, 12])

In [40]:
# sample from the model
g = torch.Generator().manual_seed(214748647 + 10)
block_size = 3

for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh( emb.view(-1,6) @ W1 + b1)
        logits = h @ W2 + b2
        #count = logits.exp()
        #probs = count / count.sum(1, keepdim=True)
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs,num_samples=1,replacement=True, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

kes.
uleuse.
tyn.
nis.
esterandurnahai.
bes.
seydatydielh.
aashdany.
ilieshuilynma.
ber.
eyse.
silyn.
anah.
sote.
yam.
secu.
kayra.
jesa.
jes.
dagh.
